In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox

# Läs in CSV-filen
df = pd.read_csv('C:/Users/theop/Downloads/csv_viewer/csv_viewer_without_data_and_filter 1/test_data.csv')


class VehicleViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Autostrada - CSV Viewer")
        self.filtered_df = df.copy()  # Save filtered dataframe here

        # Set window icon
        self.set_window_icon()

        # Style configurations
        style = ttk.Style()
        style.configure("Treeview", rowheight=30, font=('Arial', 10))
        style.configure("Treeview.Heading", font=('Arial', 12, 'bold'))
        style.configure("Treeview", background="#F5F5F5", foreground="#000000", fieldbackground="#FFFFFF")
        style.configure('TButton', font=('Arial', 10), padding=6)

        # Create a main frame
        main_frame = tk.Frame(root, bg="#EAEAEA")
        main_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)

        # Create filter frame
        filter_frame = tk.Frame(main_frame, bg="#EAEAEA")
        filter_frame.grid(row=0, column=0, sticky="ew")

        # Add a label and entry for filtering
        self.filter_label = tk.Label(filter_frame, text="Filter (Column:Value):", bg="#EAEAEA")
        self.filter_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")

        self.filter_entry = tk.Entry(filter_frame, width=40)
        self.filter_entry.grid(row=0, column=1, padx=5, pady=5)

        self.filter_button = tk.Button(filter_frame, text="Apply Filter", command=self.apply_filter)
        self.filter_button.grid(row=0, column=2, padx=5, pady=5)

        # Result Display as a table
        self.tree = ttk.Treeview(main_frame, columns=[col for col in df.columns], show='headings')
        self.tree.grid(row=1, column=0, padx=10, pady=10, sticky='nsew')

        # Scrollbars
        self.vsb = ttk.Scrollbar(main_frame, orient="vertical", command=self.tree.yview)
        self.vsb.grid(row=1, column=1, sticky='ns')
        self.tree.configure(yscrollcommand=self.vsb.set)

        self.hsb = ttk.Scrollbar(main_frame, orient="horizontal", command=self.tree.xview)
        self.hsb.grid(row=2, column=0, columnspan=2, sticky='ew')
        self.tree.configure(xscrollcommand=self.hsb.set)

        # Set column headings and initial column width
        for col in df.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100, stretch=tk.NO)  # Initial width, stretch turned off

        self.auto_adjust_column_width()

        # Row count label
        self.row_count_label = tk.Label(main_frame, text=f"Antal rader: {len(df)}", font=('Arial', 10, 'bold'), bg="#EAEAEA")
        self.row_count_label.grid(row=2, column=0, pady=10, sticky="w")

        # Configure grid weights
        main_frame.grid_rowconfigure(1, weight=1)
        main_frame.grid_columnconfigure(0, weight=1)

        # Make the main window resize with the content
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Display the full data
        self.display_results(self.filtered_df)

    def set_window_icon(self):
        try:
            # Load and set the window icon
            #self.icon_image = tk.PhotoImage(file='icon.png')  # Path to your icon file
            #self.root.iconphoto(True, self.icon_image)
            pass  # Do nothing in case there is no icon
        except tk.TclError:
            # Handle error if icon cannot be loaded
            print("Error loading icon image. Make sure the file path is correct and the file is a valid image format.")

    def auto_adjust_column_width(self):
        # Create a label for text measurement
        label = tk.Label(self.root, font=('Arial', 10))
        max_width = {col: len(self.tree.heading(col, 'text')) * 10 for col in self.tree["columns"]}  # Start with heading width

        # Check all column values
        for item in self.tree.get_children():
            row_values = self.tree.item(item, 'values')
            for col, value in zip(self.tree["columns"], row_values):
                label.config(text=value)
                text_width = label.winfo_reqwidth()
                max_width[col] = max(max_width[col], text_width)

        # Set column width based on the widest content
        for col, width in max_width.items():
            self.tree.column(col, width=width + 20)  # Add extra padding

    def display_results(self, df_to_display):
        # Clear previous results
        for item in self.tree.get_children():
            self.tree.delete(item)

        # Display results
        for _, row in df_to_display.iterrows():
            self.tree.insert("", "end", values=row.tolist())

    def apply_filter(self):
        filter_text = self.filter_entry.get()
        if not filter_text:
            self.filtered_df = df.copy()  # If filter is empty, reset to full DataFrame
        else:
            try:
                column, value = filter_text.split(':', 1)
                column = column.strip()
                value = value.strip()
                if column in df.columns:
                    self.filtered_df = df[df[column].astype(str).str.contains(value, case=False, na=False)]
                else:
                    messagebox.showwarning("Invalid Column", f"Column '{column}' does not exist.")
                    self.filtered_df = df.copy()
            except ValueError:
                messagebox.showwarning("Invalid Filter", "Filter format should be 'Column:Value'.")
                self.filtered_df = df.copy()

        # Update the display with filtered data
        self.display_results(self.filtered_df)
        self.row_count_label.config(text=f"Antal rader: {len(self.filtered_df)}")

    def export_to_csv(self):
        # Ask user for a file name and location
        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                 filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if not file_path:
            return  # User canceled the save dialog

        try:
            # Export DataFrame to CSV with UTF-8 encoding
            self.filtered_df.to_csv(file_path, index=False, encoding='utf-8-sig')
            messagebox.showinfo("Export Success", "Data exported successfully!")
        except Exception as e:
            messagebox.showerror("Export Error", f"An error occurred while exporting the data:\n{e}")


# Create the application window
root = tk.Tk()
app = VehicleViewerApp(root)
root.mainloop()


In [15]:
%run "C:/Users/theop/Downloads/csv_viewer/csv_viewer_without_data_and_filter 1/csv_viewer_without_data_and_filter.py"

In [ ]:
#FRÅGA 1
# Det här programmet använder tkinter för att skapa ett grafiskt fönster där man kan kolla på data från en CSV-fil och filtrera dem. Först så läser programmet in CSV-filen, som heter test_data.csv, med hjälp av pandas. All data läggs in i en tabell. 
# Tabellen visas i ett fönster som öppnas när man kör programmet. Själva fönstret görs med hjälp av en klass som heter VehicleViewerApp och där finns en tabell där man kan se alla rader från CSV-filen. Man kan också använda ett filter för att bara visa vissa rader. Till exempel, om man vill se alla rader från år 2012, så skriver man year:2012 i fältet och trycker på 'Apply Filter'. Då visas bara de raderna.
# Om man vill ta bort filtret och visa allt igen, lämnar man fältet tomt och trycker på knappen igen, och då visas alla rader från filen igen.
# Förutom det, finns det lite kod för att styra hur fönstret ser ut, till exempel hur texten och knapparna ser ut och fungerar. Så det programmet gör är att visa data från CSV-filen i en tabell och låta användaren filtrera datan på ett enkelt sätt.


In [9]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 1
a = 10
b = [5, 7, 3]
c = {2, 7, 1, 8, 2, 8}
def my_fun(x):
    return 2 * 2
d = my_fun

print(type(a))  # int
print(type(b))  # list
print(type(c))  # set
print(type(d))  # function 


<class 'int'>
<class 'list'>
<class 'set'>
<class 'function'>


In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 2
my_variable = (1, 1, 2, 3, 5)

print(isinstance(my_variable, tuple))  # True
print(isinstance(my_variable, list))   # False


In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 3
# a) Skapa klassen FruitProduct med en docstring
class FruitProduct:
    """A class representing fruit products in a grocery store"""
    
    # b) Definiera instansattributen med __init__
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity

# c) Skapa en instans av klassen
swedish_apples = FruitProduct(52, 1)

# d) Skriv ut docstring för klassen
class_docstring = FruitProduct.__doc__

# e) Skriv ut attributen för instansen
apple_price = swedish_apples.price
apple_quantity = swedish_apples.quantity

# f) Kontrollera typen av instansen
instance_type = type(swedish_apples)

# g) Kontrollera om instansen tillhör klassen FruitProduct
is_instance = isinstance(swedish_apples, FruitProduct)

class_docstring, apple_price, apple_quantity, instance_type, is_instance



In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 4
class Square:
    def __init__(self, side_length):
        self.side_length = side_length

    def perimeter(self):
        return 4 * self.side_length

    def area(self):
        return self.side_length ** 2

my_square = Square(8)
print(my_square.perimeter())  # 32
print(my_square.area())       # 64



In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 5
class DescriptiveStatistics:
    """Denna klass ger funktioner för att beräkna beskrivande statistik från en lista."""
    def __init__(self):
        self.data = []

    def add_data(self, data):
        if isinstance(data, list):
            self.data.extend(data)
        else:
            raise Exception('Endast "listor" accepteras som data.')

    def calc_sum(self):
        return sum(self.data)

    def calc_nbr_of_elements(self):
        return len(self.data)

    def calc_mean(self):
        return self.calc_sum() / self.calc_nbr_of_elements()

    def print_summary(self):
        print('Summa:', self.calc_sum())
        print('Antal element:', self.calc_nbr_of_elements())
        print('Medelvärde:', self.calc_mean())

# Lista över data
L = [1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1]
my_data = DescriptiveStatistics()
my_data.add_data(L)

# Skriv ut data och statistik
print('Summa:', my_data.calc_sum())
print('Antal element:', my_data.calc_nbr_of_elements())
print('Medelvärde:', my_data.calc_mean())
my_data.print_summary()


# a) Klassen DescriptiveStatistics är ett verktyg som hjälper för att göra grundläggande beräkningar på listor med siffror. När man skapar den börjar den med en tom lista. Man kan lägga till data i listan med metoden add_data() men det måste vara en lista (om man försöker lägga till något annat kommer det att ge ett felmeddelande).
# Sedan finns det några användbara funktioner:
# calc_sum() räknar ut summan av alla siffror i listan.
# calc_nbr_of_elements() räknar hur många element som finns i listan.
# calc_mean() beräknar medelvärdet, alltså summan delat med antalet element.
# Till sist med print_summary() kan man skriva ut alla dessa resultat tillsammans (summan, antalet element och medelvärdet).
# Så om man har en lista med siffror kan den här klassen hjälpa för att räkna ut hur mycket alla siffror blir tillsammans, hur många de är och vad deras medelvärde är.


# b) Om vi ändrar variabeln L till en tuple i stället för en lista:
# L = (1, 2, 1, 3, 5, 7, 4, 9, 10, 3, 2, 1, 6, 4, 3, 2, 1, 10, 9, 1, 8, 7, 3, 2, 1)
# då vid körning av programmet kommer ett exception att kastas med meddelandet:
# Exception: Endast "listor" accepteras som data.



In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 6
# Uppgiften handlar om att skapa en klass som fungerar som ett bankkonto. Det vi ska göra är att skapa en klass med två attribut: namnet på kontoinnehavaren och kontots saldo. Sedan ska vi skapa två grundläggande funktioner (metoder): en för att sätta in pengar på kontot och en för att ta ut pengar.
# När vi gör ett uttag, kommer vi att kontrollera om summan vi vill ta ut är större än saldot på kontot. Om det är det, visas ett meddelande om att saldot är för lågt och uttaget kommer inte att genomföras. Annars dras pengarna från kontot.
# Slutligen kan vi skapa ett konto, sätta in pengar, göra uttag och se vad som händer om vi försöker ta ut mer pengar än vad som finns på kontot.


class BankAccount:
    def __init__(self, account_holder, balance=0):
        self.account_holder = account_holder  # Kontoinnehavarens namn
        self.balance = balance  # Kontots saldo

    def deposit(self, amount):
        self.balance += amount  # Lägger till beloppet till saldot
        print(f"{amount} kr insatta. Nytt saldo: {self.balance} kr")

    def withdraw(self, amount):
        if amount > self.balance:
            print("För lågt saldo!")
        else:
            self.balance -= amount  # Drar beloppet från saldot
            print(f"{amount} kr uttagna. Nytt saldo: {self.balance} kr")

# Skapa ett konto för en kund
my_account = BankAccount("Maria", 200)

# Testa insättningar och uttag
print(f"Kontoinnehavare: {my_account.account_holder}")
print(f"Startsaldo: {my_account.balance} kr")
my_account.deposit(50)  # Sätta in pengar
my_account.withdraw(20)  # Ta ut pengar
my_account.withdraw(250)  # Försök ta ut mer än tillgängligt saldo


# Vi skapar ett konto för "Maria" med ett startsaldo på 200 kr.
# Vi sätter in 50 kr, så saldot blir 250 kr.
# Vi tar ut 20 kr, vilket lämnar 230 kr.
# Vi försöker ta ut 250 kr men eftersom saldot är för lågt visas ett meddelande om att det inte finns # tillräckligt med pengar.




In [ ]:
#FRÅGA 2, KAPITEL 6 UPPGIFT 7

# Jag ska svara så här: En klass är som en "ritning" för att skapa objekt. Man kan tänka på det som en mall, till exempel ritningen av en bil. Man kan skapa många bilar från samma ritning men varje bil är en instans (alltså en kopia eller ett objekt) av den klassen.
# När vi skapar en klass ger vi den några attribut (egenskaper), alltså information som varje objekt 
# kommer att ha. Om klassen är en bil kan attributen vara bilens färg, hastighet och så vidare. Vi kan
# också skapa metoder, alltså saker som objektet kan göra. Om objektet är bilen, kan en metod vara 
# "starta", "stanna" eller "gasa".
# Så kort sagt, klassen är ritningen, attributen är egenskaperna och metoderna är funktionerna som  
# objektet kan utföra.


In [ ]:
#FRÅGA 2, KAPITEL 7 UPPGIFT 1

# a) Om man jobbar på ett projekt där det finns interna riktlinjer för kodstil som går emot PEP 8, så ska  man följa de interna riktlinjerna. Även om PEP 8 är en bra standard kan olika projekt ha sina egna  specifika regler som gäller.
# b) Kommentarer i koden ska generellt sett skrivas på engelska enligt PEP 8. Även om koden själv är på ett annat språk är engelska bäst för att alla ska kunna förstå det internationellt.
# c) Funktioner och variabler ska namnges med snake_case. Det betyder små bokstäver och att ord separeras med understreck (_).
# d) Klasser ska namnges med PascalCase. Det innebär att varje ord börjar med stor bokstav och att det inte finns några mellanrum eller understreck.
# e) Importer av moduler ska alltid placeras högst upp i filen, direkt efter eventuella kommentarer eller docstrings.
# f) Enligt PEP 8 kan man använda antingen enkla ('min_sträng') eller dubbla ("min_sträng") citattecken för att skapa en sträng. Det viktiga är att vara konsekvent i hela projektet och använda samma stil genomgående.
# g) Alternativ 1 är korrekt enligt PEP 8 eftersom det ska finnas mellanslag runt operatorer som används för att tilldela och räkna med tal.



In [ ]:
#FRÅGA 2, KAPITEL 7 UPPGIFT 2

# a) Frasen ”Explicit is better than implicit” betyder att det är bättre att skriva kod som är tydlig och lätt att förstå även om den ibland kan vara lite längre. Det handlar om att inte försöka förkorta eller gömma saker i koden på ett sätt som gör den svårare att läsa. Ett exempel kan vara att ge variabler tydliga namn istället för korta och oklara namn eller att skriva ut hela villkor tydligt istället för att försöka göra dem för kompakta.
# b) Frasen ”Simple is better than complex” innebär att enkel kod är bättre än komplicerade lösningar, så länge den gör jobbet. Det betyder att om man kan välja mellan en enkel lösning och en mer avancerad bör man välja den enkla. Till exempel, en enkel for-loop är ofta bättre än en komplicerad lösning med många inbyggda funktioner om de båda ger samma resultat.


In [ ]:
#FRÅGA 2, KAPITEL 7 UPPGIFT 3

# När vi skriver kod är det sant att alla kan skriva på sitt eget sätt men det kan skapa problem, särskilt när vi måste samarbeta med andra utvecklare eller om någon annan behöver förstå vår kod senare. PEP 8 och liknande regler hjälper till att hålla koden ren och enhetlig så att alla kan förstå den lättare.
# Till exempel, om alla skriver kod på samma sätt blir det enklare att underhålla koden och hitta fel. Även om någons program fungerar som han vill kan det skapa problem i framtiden om koden är rörig eller svår att läsa.


In [ ]:
#FRÅGA 2, KAPITEL 8 UPPGIFT 1

# a) Syntaxfel är när koden vi skriver inte följer språkets regler. Dessa fel upptäcks innan programmet körs. Till exempel, om du glömmer att stänga en parentes eller skriver fel på en kommando får du ett syntaxfel. För att programmet ska kunna köras måste du rätta till dessa fel.
# b) Exceptions är ett sätt att hantera oväntade situationer eller fel utan att programmet kraschar. Till exempel, om du försöker dela med noll skulle programmet krascha om du inte fångar upp undantaget. Genom att hantera undantag kan du ge användaren ett meddelande om vad som gick fel och låta programmet fortsätta köra istället för att stanna.
# c) När du gör raise av ett undantag (exception), stoppar du programmet för att ett allvarligt fel har inträffat som du inte kan hantera just då. Att använda raise är som att säga "Här är ett problem, vi kan inte fortsätta". Det hjälper till att hantera fel på rätt sätt och ser till att programmet inte fortsätter i en felaktig situation.


In [ ]:
#FRÅGA 2, KAPITEL 8 UPPGIFT 2

def convert_string_to_int(string):
    try:
        int(string)
    except ValueError:
        return "Invalid input, cannot convert to integer."
    else:
        return int(string)

print(convert_string_to_int("314"))
print(convert_string_to_int("abc"))


# a) Funktionen convert_string_to_int försöker konvertera en sträng till ett heltal (int).
# Den använder try för att försöka göra konverteringen. Om konverteringen misslyckas (om strängen innehåller något som inte kan omvandlas till ett tal, som bokstäver) så går programmet in i except-blocket och returnerar meddelandet "Invalid input, cannot convert to integer."
# Om konverteringen lyckas går programmet inte in i except, utan kör else-blocket och returnerar det omvandlade heltalet.
# Så: När man anropar funktionen med strängen "314" returnerar den 314 eftersom den kan omvandlas till ett heltal.
# När man anropar funktionen med strängen "abc" returnerar den felmeddelandet eftersom den inte kan omvandla bokstäver till siffror.
# b) Else används här för att köras endast om try-blocket lyckas, alltså om konverteringen går igenom utan problem. Det skiljer den normala programmets körning från felhanteringen. Om konverteringen misslyckas körs inte else och programmet hoppar direkt till except. Med else gör man koden tydligare: om det inte finns något fel kör det som finns i else.


In [ ]:
#FRÅGA 2, KAPITEL 8 UPPGIFT 3

# När vi skriver kod kan det uppstå situationer där något går fel, som när vi försöker dela ett tal med noll eller när vi ber användaren att mata in ett tal och de istället skriver bokstäver. I dessa fall kommer programmet att sluta köra om vi inte fångar felet.
# Ett exempel på hur vi kan fånga dessa fel (exceptions) är följande:

try:
    number = int(input("Skriv in ett tal: "))
    result = 10 / number
    print(f"Resultatet är: {result}")
except ValueError:
    print("Var god och skriv in ett giltigt tal!")
except ZeroDivisionError:
    print("Du kan inte dela med noll!")

    
    
# I detta exempel:
# Först ber vi användaren att mata in ett tal.
# Om användaren skriver in något som inte kan omvandlas till ett tal, till exempel bokstäver, kommer programmet att fånga ValueError och visa meddelandet "Var god och skriv in ett giltigt tal!".
# Om användaren skriver in siffran 0, kommer programmet att fånga ZeroDivisionError och visa meddelandet "Du kan inte dela med noll!".
    


In [ ]:
#FRÅGA 2, KAPITEL 8 UPPGIFT 4

def add_two_small_numbers(a, b):
    if a > 100 eller b > 100:
        raise ValueError("Båda talen måste vara mindre än eller lika med 100.")
    return a + b

try:
    result = add_two_small_numbers(50, 120)  # Testa med ett tal över 100
    print(f"Resultatet är: {result}")
except ValueError as e:
    print(e)

    
# I det här exemplet:
# Funktionen add_two_small_numbers kontrollerar först om a eller b är större än 100.
# Om de är det kastar den ett ValueError och visar meddelandet "Båda talen måste vara mindre än eller lika med 100".
# Vi använder try-except för att fånga felet och visa det.
    

In [ ]:
#FRÅGA 2, KAPITEL 8 UPPGIFT 5

# I det första kodstycket:

try:
    5 + "Python is fun!"
except Exception as exception_instance:
    print(type(exception_instance))
    print(exception_instance)

# Här försöker hon att lägga ihop en siffra (5) med en sträng ("Python is fun!"). Det kommer att orsaka ett TypeError eftersom man inte kan lägga ihop en siffra med en text. Koden fångar felet och skriver ut vilken typ av fel det är och själva felmeddelandet.

# I det andra kodstycket:

try:
    5/0
except Exception as exception_instance:
    print(type(exception_instance))
    print(exception_instance)

# Här försöker hon att dela 5 med 0 vilket kommer att orsaka ett ZeroDivisionError. Koden fångar undantaget och skriver ut vilken typ av fel det är och felmeddelandet.

# I det tredje kodstycket finns en funktion som försöker dela två tal:

def add_two_numbers(a, b):
    try:
        return(a/b)
    except TypeError:
        print("Båda argumenten måste vara nummer.")
    except ZeroDivisionError:
        print("Division med noll är inte tillåten.")

        
# Här hanterar funktionen två typer av undantag: TypeError, som inträffar om ett av argumenten inte är ett nummer, och ZeroDivisionError, som inträffar när man försöker dela med noll.  

# Till sist testar hon funktionen med olika inmatningar:

print(add_two_numbers(5, 2))     # Vanlig division
print(add_two_numbers(5, "hello")) # Orsakar TypeError
print(add_two_numbers(5, 0))      # Orsakar ZeroDivisionError

# Här testar hon funktionen för att se om felen fångas korrekt. Det första exemplet fungerar som det ska, medan de andra två orsakar fel som fångas av undantagshanteringen och skriver ut ett felmeddelande.

In [ ]:
#FRÅGA 2, KAPITEL 9 UPPGIFT 1

# Att bygga tester för din kod är viktigt eftersom det hjälper dig att säkerställa att din kod fungerar som den ska och att inga oväntade problem uppstår. När du lägger till eller ändrar kod kan testerna visa om något bryts utan att du märker det. Det hjälper också andra utvecklare som arbetar med dig att förstå vad koden gör och säkerställer att resultaten är vad du förväntar dig.

In [12]:
#FRÅGA 2, KAPITEL 9 UPPGIFT 2

# Funktionen multiply_two_numbers multiplicerar två tal och returnerar resultatet. Vi kan skapa några testexempel med doctest.

def multiply_two_numbers(a, b):
    """
    Returnerar produkten av två tal, a och b.

    >>> multiply_two_numbers(2, 3)
    6
    >>> multiply_two_numbers(5, 0)
    0
    >>> multiply_two_numbers(-1, 3)
    -3
    >>> multiply_two_numbers(2.5, 4)
    10.0
    """
    return a * b

if __name__ == "__main__":
    import doctest
    doctest.testmod()

    
# Med det här exemplet kommer doctest att köra de exempel vi har lagt till i kommentarerna (docstring) för att säkerställa att funktionen fungerar korrekt. Och om vi vill print ut resultat har vi följande koden:

def multiply_two_numbers(a, b):
    """
    Return the product of a and b.

    >>> multiply_two_numbers(2, 3)
    6
    >>> multiply_two_numbers(5, 0)
    0
    >>> multiply_two_numbers(-1, 3)
    -3
    >>> multiply_two_numbers(2.5, 4)
    10.0
    """
    return a * b

# Den fungar bättre i Jupyter
import doctest
doctest.testmod(verbose=True)




Trying:
    multiply_two_numbers(2, 3)
Expecting:
    6
ok
Trying:
    multiply_two_numbers(5, 0)
Expecting:
    0
ok
Trying:
    multiply_two_numbers(-1, 3)
Expecting:
    -3
ok
Trying:
    multiply_two_numbers(2.5, 4)
Expecting:
    10.0
ok
9 items had no tests:
    __main__
    __main__.VehicleViewerApp
    __main__.VehicleViewerApp.__init__
    __main__.VehicleViewerApp.apply_filter
    __main__.VehicleViewerApp.auto_adjust_column_width
    __main__.VehicleViewerApp.display_results
    __main__.VehicleViewerApp.export_to_csv
    __main__.VehicleViewerApp.set_window_icon
    __main__.my_fun
1 items passed all tests:
   4 tests in __main__.multiply_two_numbers
4 tests in 10 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)

In [ ]:
#FRÅGA 2, KAPITEL 9 UPPGIFT 3

# Här skriver jag en kod med pytest.Vi kan använda pytest för att skapa tester. Här är ett exempel:

import pytest
from my_module import multiply_two_numbers

def test_multiply_two_numbers():
    assert multiply_two_numbers(2, 3) == 6
    assert multiply_two_numbers(5, 0) == 0
    assert multiply_two_numbers(-1, 3) == -3
    assert multiply_two_numbers(2.5, 4) == 10.0

if __name__ == "__main__":
    pytest.main()






In [ ]:
#FRÅGA 2, KAPITEL 10 UPPGIFT 1

# En modul är en Python-fil som innehåller kod som funktioner, variabler eller klasser och som kan importeras och användas i andra filer. Till exempel, om man har skapat funktioner i en fil som heter mymodule.py  kan man importera dem till en annan fil med kommandot import mymodule och använda funktionerna där.

In [ ]:
#FRÅGA 2, KAPITEL 10 UPPGIFT 2

# a)Importera modulen math från Python-biblioteket utan alias. Här importerar vi modulen math utan att ge den ett alias (en förkortning). Då kan vi använda alla funktioner i modulen som de är.

import math


# b)Använd kommandot help() för funktionen math.ceil(). Funktionen math.ceil() rundar av ett decimaltal uppåt till närmaste heltal. Den är användbar när man vill få nästa heltal från ett decimaltal.

help(math.ceil)


# c)Exekvera math.ceil(10.2) och förklara resultatet. Resultatet är 11 eftersom funktionen ceil() rundar upp 10.2 till närmaste heltal, vilket är 11.

import math
print(math.ceil(10.2))


# d)Exekvera math.ceil(9.8) och förklara resultatet. Resultatet är 10 eftersom 9.8 rundas upp till 10, vilket är det närmaste heltalet.

import math
print(math.ceil(9.8))





In [ ]:
#FRÅGA 2, KAPITEL 10 UPPGIFT 3

# Jag skapar en modulfil (module), en Python-fil, kallas mymodule.py. Denna fil kommer att vara min modul och I den filen skriver jag två funktioner.

# mymodule.py

def add(a, b):
    """Adderar två tal och returnerar resultatet."""
    return a + b

def subtract(a, b):
    """Subtraherar det andra talet från det första och returnerar resultatet."""
    return a - b

# sparar det i min dator. Jag skapar en annan Python-fil, kallas main_script.py, som ska importera min modul och använda funktionerna.

# main_script.py

# Importera modulen (mymodule)
import mymodule

# Använd funktionen add
result_add = mymodule.add(10, 5)
print(f"Resultatet av additionen är: {result_add}")

# Använd funktionen subtract
result_subtract = mymodule.subtract(10, 5)
print(f"Resultatet av subtraktionen är: {result_subtract}")

# Jag sparar båda filerna (mymodule.py och main_script.py) i samma mapp på min dator.

# Kör main_script.py. När jag kör det kommer denna fil att importera mymodule.py och använda de funktioner jag definierade där.

In [ ]:
#FRÅGA 2, KAPITEL 11 UPPGIFT 1

# Loggfiler är mycket viktiga för företag och organisationer eftersom de hjälper till att övervaka system och applikationer. De ger detaljer om fel, varningar, framgångar och annan användbar information som hjälper tekniker att identifiera och lösa problem. Utan loggfiler är det svårt att veta vad som gick fel och när.

In [ ]:
#FRÅGA 2, KAPITEL 11 UPPGIFT 2

# I loggfiler finns det olika nivåer av meddelanden som bestämmer allvaret eller betydelsen av händelserna som loggas. De vanligaste nivåerna är:
# DEBUG: Detaljerad information för felsökning.
# INFO: Allmän information om normal systemdrift.
# WARNING: Varningar om potentiella problem som ännu inte har orsakat fel.
# ERROR: Fel som orsakar problem för programmet men inte nödvändigtvis avslutar det.
# CRITICAL: Allvarliga fel som kan avsluta programmet.


In [ ]:
#FRÅGA 2, KAPITEL 11 UPPGIFT 3

# jag skapade ett enkelt program som använder logging-modulen för att logga meddelanden på olika nivåer. Detta program loggar olika meddelanden på olika nivåer och skriver informationen till filen my_log.log. Om ett fel uppstår vid division, loggas det som ERROR.

import logging

# Konfigurera loggningen för att skriva till en loggfil
logging.basicConfig(filename='my_log.log', level=logging.DEBUG)

# Olika loggnivåer
logging.debug("Detta är ett debug-meddelande.")
logging.info("Detta är ett informationsmeddelande.")
logging.warning("Detta är en varning!")
logging.error("Detta är ett fel.")
logging.critical("Detta är ett kritiskt fel.")

# En enkel funktion med felhantering och loggning
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        logging.error("Division med noll!")
        return "Fel: Du kan inte dela med noll."

# Testa funktionen
print(divide(10, 2))  # Normal körning
print(divide(10, 0))  # Fel vid division med noll
